# SABAT SMFI

- Describes SABAT program

In [5]:
%matplotlib inline
from IPython.display import Image
from IPython.display import Math
from IPython.display import Latex

In [6]:
%load_ext autoreload
%autoreload 2
import os
import sys
import time
import warnings
import datetime 
print("Last updated on ", time.asctime())

Last updated on  Mon Jun 22 19:31:25 2020


In [7]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib
import tables            as tb
import random
import glob
import warnings
sns.set()
import matplotlib.pyplot as plt


In [8]:
from dataclasses import dataclass

In [9]:
from  invisible_cities.core.system_of_units import *
from  invisible_cities.core.system_of_units import MHZ

In [10]:
from sabat.sbt_types import FoV, CircularFoV
from sabat.sbt_types import Laser, PulsedLaser, GLaser, GaussianBeam
from sabat.sbt_types import Molecule
from sabat.sbt_types import Molecule2P
from sabat.sbt_types import DyeSample
from sabat.sbt_types import Microscope
from sabat.sbt_types import CCD

In [11]:
from sabat.sbt_core import  power_density
from sabat.sbt_core import  photon_density
from sabat.sbt_core import  fluorescence_per_molecule
from sabat.sbt_core import  duration_of_fluorescence
from sabat.sbt_core import  diffraction_limit
from sabat.sbt_core import  photoelectrons_per_pixel
# from sabat.sbt_core import  fluorescence_2p
from sabat.sbt_core import  absorbed_photons_per_fluorophore_per_pulse_2p

# SABAT SMFI

## Authors

- J.J. Gomez-Cadenas 

[NEXT](https://next.ific.uv.es/next/)

#### Contact


- [Email](mailto:jjgomezcadenas@dipc.org)




# S/N in a monolayer target

In [8]:
photon = 1
molecule = 1
GM = 1e-50 * cm2*cm2*second / (photon * molecule)
us = photon / second
ucm2 = photon / cm2
ucm3 = molecule / cm3
gp = 0.664

## NEXT PRL

### Molecule

In [9]:
fl = Molecule(name = 'Fluorescein', sigma=3E-16 * cm2, Q=0.9)
fl


        Molecule name =Fluorescein; cross section = 3.0e-16 cm2; Q =   0.9
        

### Laser

In [11]:
lb = Laser(lamda=488*nm, power=1 * microwatt)  
lb


        Laser:
        wavelength                =4.9e+02 nm
        photon energy             =2.5e+00 eV
        power                     =1.0e-03 mW
        energy per second         =1.0e-03 mJ
        photons per second        =2.5e+12 ph/second
        

### Microscope

In [12]:
mc = Microscope(name='UTA', numerical_aperture=1.4, magnification=100, eff_dichroic = 0.85, eff_filter = 0.8) #0.2 for QE of PMT
print(mc)


        name                 = UTA
        NA                   =   1.4
        M                    = 100.0
        eff dichroic         =  0.85
        eff filter           =  0.80
        Optical transmission =  0.30
        Filter  transmission =  0.68
        Total transmission   =  0.20
        


### FOV

In [13]:
fov= CircularFoV(d = 35 * micron, z= 0.1 * micron)
fov


        FOV; d = 3.5e-02 mm; r = 1.8e-02 mm; z = 1.0e-04 mm;
        area = 9.6e-04 mm2 volume = 9.6e-08 mm3
        

In [14]:
I = photon_density(lb, fov)
print(f'photon density = {I/(us*ucm2):5.1e} photons/(second cm2)')

photon density = 2.6e+17 photons/(second cm2)


### Dye concentration

In [15]:
ds = DyeSample(name='fp',concentration = 1 * nanomole/l, volume= fov.volume())
ds


        Dye name =fp;
        concentration = 1.0e-09 mole/l (6.0e+11 molecules/cm3);
        V = 9.6e-14 l,
        nof molecules = 5.8e+01
        

### Fluorescence per molecule

In [17]:
f = fluorescence_per_molecule(fl, I)
print(f'Fluorescence per molecule and second = {f/us:5.1e}')

Fluorescence per molecule and second = 6.9e+01


In [19]:
F = f * ds.n_molecules()
print(f'Fluorescence per FOV and second = {F/us:5.1e}')

Fluorescence per FOV and second = 4.0e+03


## The monolayer case.

In [95]:
@dataclass
class Monolayer(FoV):
    ns     : float
    nb     : float

    def n_molecules(self)->float:
        return self.area() / nm**2

    def snr(self)->float:
        return self.ns /np.sqrt(2 * self.nb * self.n_molecules())

    def SNR(self, repRate : float)->float:
        return self.snr() * np.sqrt(repRate)

    def nobs(self)->float:
        s = np.random.normal(self.ns, np.sqrt(self.ns))
        b = np.random.normal(self.nb * self.n_molecules(), np.sqrt(self.nb * self.n_molecules()))
        return s + b 

    def nsignal(self)->float:
        return self.nobs() - self.nb * self.n_molecules()

    def __repr__(self):
        s ="""
        
        x = {0:5.1e} mum; y = {1:5.1e} mum; z = {2:5.1e} nm; area = {3:5.1e} mum2 volume = {4:5.1e} mum3
        nof molecules = {5:5.1e}
        snr           = {6:5.1e}
        """.format(self.x/mum, self.y/micron, self.z/nm, self.area()/micron2, self.volume()/micron3, self.n_molecules(), self.snr())

        return s


In [96]:
ml = Monolayer(x=10*micron, y=10*micron, z = 1*nm, ns = 100, nb = 1)
ml


        
        x = 1.0e+01 mum; y = 1.0e+01 mum; z = 1.0e+00 nm; area = 1.0e+02 mum2 volume = 1.0e-01 mum3
        nof molecules = 1.0e+08
        snr           = 7.1e-03
        

In [97]:
R = 1e+6
print(f'SNR for a repetition rate of {R:5.1e} = {ml.SNR(R):5.1e}')

SNR for a repetition rate of 1.0e+06 = 7.1e+00


In [98]:
I = photon_density(lb, ml)
print(f'photon density = {I/(us*ucm2):5.1e} photons/(second cm2)')

photon density = 1.2e+24 photons/(second cm2)


In [99]:
print(f'photon density per pulse= {I/lb.tau/(us*ucm2):5.1e} photons/(second cm2)')

photon density per pulse= 1.2e+28 photons/(second cm2)


In [100]:
f = fluorescence_per_molecule(fl, I)
print(f'Fluorescence per molecule and second = {f/us:5.1e}')

Fluorescence per molecule and second = 3.3e+08


In [101]:
print(f'Fluorescence per molecule and pulse = {(f/lb.f):5.1e}')

Fluorescence per molecule and pulse = 3.3e+00


#### However we can absorb at max 1 photon per pulse. 

In [102]:
n_f = n_pulses_per_second = lb.f/hertz
print(f' number of absorbed photons per fluorophore per second = {n_pulses_per_second:5.1e}')

 number of absorbed photons per fluorophore per second = 1.0e+08


In [105]:
n_ph = n_f * 1e-6
print(f' number of photons produced per microsecond = {n_ph:5.1e}')
print(f' number of photons recorded per microsecond = {n_ph * 0.1:5.1e}')

 number of photons produced per microsecond = 1.0e+02
 number of photons recorded per microsecond = 1.0e+01


### Detection efficiency

- Order 10-20 %. 
- Suppose that one wants to record at least 10 photons per sample. This corresponods to 100 photons produced, thus R~1E+6 in one second.
- One needs a second to scan one point of 10 x 10 micron2

### Photobleaching

In [106]:
n_epb = 36000
df = (n_epb/n_f)
print(f'duration of fluorescence = {df * 1e+6:5.1e}  mus')

duration of fluorescence = 3.6e+02  mus


### This means that R = 360 rather than 1E+6. 

- Photobleaching can increase by one order of magnitude in deoxygenated environment, but this is still not eough to compensate. 

## Small scans.

- The natural "size" of the 2p spot is about 1 micron. 

In [107]:
gb = GaussianBeam(laser=lb, mc = mc)  
gb


        w0                   =   0.1 micron
        zr                   =   0.1 micron
        DOF                  =   0.2 micron
        

In [47]:
print(f'photons observed (one shot) = {ml.nobs():5.1e}, signal (one shot) =  {ml.nsignal():5.1e}')

photons observed (one shot) = 1.0e+08, signal (one shot) =  3.3e+03


In [49]:
np.average([ml.nsignal() for i in range(1000000)])

112.7505127848183

### Consider a pulsed beam

In [67]:
lb = PulsedLaser(lamda=488*nm, power=500*mW, f=100*MHZ, tau=100*femtosecond)  
lb


        Pulsed Laser:
        wavelength                =4.9e+02 nm
        photon energy             =2.5e+00 eV
        power                     =5.0e+02 mW
        repetition rate           =1.0e+02 kHz
        pulse width               =1.0e+02 fs
        energy per pulse          =5.0e+04 fJ
        energy per second         =5.0e+02 mJ
        photons per second        =1.2e+18 ph/second
        

### Assume a FOV of 10 x 10 microns2 times 1 nm

In [68]:
fov= CircularFoV(d = 10 * micron, z= 1 * nm)
fov


        FOV; d = 1.0e-02 mm; r = 5.0e-03 mm; z = 1.0e-06 mm;
        area = 7.9e-05 mm2 volume = 7.9e-11 mm3
        

In [69]:
565*12

6780

## State of the art: Molecule

- F. Cossio and his group have developed a new molecule (C826)
- Measurements with spectrofluoremeter show that the chelated molecule shines with a peak at 425 nm when excited at 250 nm. The unchelated molecule response at that wavelength is rather week. 
- The unchelated molecule shines at 489 nm for all excitations, with higher response when excited with a blue laser (436 nm).

## C826

<img align="center" src="img/dipcCrown_8_2_6.png" width="600" height="600"></img>

- 8+2+6 Crown     : C826, developed by F. Cossio group. 
- Excitation light: 250 nm results in maximum separation between chelated and unchelated molecules


## Laser for UV excitation (chelated molecule):

https://www.rpmclasers.com/product/p4-235-5-2-0-5-236nm-microchip-laser/

https://rpmclasers.com/wp-content/uploads/products/Multi-Wavelength%20Series%20v1_1.pdf

- Wavelength      : 236 nm 
- Beam power      : 1 mW
- Beam energy     : 0.5 muJ per pulse
- Pulse width     : 1.4 nm 
- Beam rate.      : 1 - 10 kHz

- Quote: The price for this laser is 16,173 and has a lead time of 12 weeks ARO.  This price includes the laser head, controller w/ software, power supply and heatsink, so it is everything you need to run the laser right out of the box. 


- STU-01E-100 laser

https://www.gophotonics.com/products/lasers/teem-photonics/29-689-stu-01e-1x0

- Wavelength  : 266 nm 
- Beam power  : 2 mW
- Beam energy : 1 muJ per pulse
- Pulse width : 0.4 ns
- Beam rate.  : 4 kHz

- Quote: About the cost :
- STU-01E-1xx laser unit 8 957 euro.
- MLC-03A-DP1 controller unit 752 euro.
- External trigger (3x) RR mode option xxx–x3x : 299 euro.
- Note: other rep. rate mode options are not charge, ie. 0 euro.
- Lead time : about 8-10 weeks.



## SABAT-SMFI (wet)

### Beam

In [4]:
laser = PulsedLaser(lamda=266*nm, power=2*mW, f=4*MHZ, tau=100*femtosecond)  

print(laser)
fov = FoV(x = 35 * micron, y = 35 * micron, z = 2 * 488 * nm)
print(fov)
print(f'power density for laser focused in FOV  = {power_density(laser,fov)/(W/cm2):5.1f} W/cm2')
I = photon_density(laser, fov, 1*second)
print(f'photon density for UV laser FOV (per pulse of 1 s)  = {I/(1/cm2):5.1e} ph per cm2')



NameError: name 'nm' is not defined

### Molecule

In [86]:
m = Molecule(name='C826', sigma=9E-17 * cm2, Q = 0.8)  #guess
print(m)
flm = fluorescence_per_molecule(m, I, 1.0*second)
print('Fluorescence per molecule and pulse 1  second = {0:5.1e}'.format(flm))




        Molecule name =C826; cross section = 9.0e-17 cm2; Q =   0.8
        
Fluorescence per molecule and pulse 1  second = 1.6e+04


### SABAT Microscope

- The microscope for the dry experiment will operate in vacuum. Numerical aperture is limited to less than 1. We assume 0.7

In [89]:
mc = Microscope(name='SABAT', numerical_aperture=0.7, magnification=100)
print(mc)


        name                 = SABAT
        NA                   =   0.7 
        M                    = 100.0 
        eff dichroic         =  0.85 
        eff filter           =  0.80 
        Optical transmission =  0.07
        Filter  transmission =  0.68
        Total transmission   =  0.05
        


In [90]:
fov = FoV(x = 0.8 * micron, y = 0.8 * micron, z = 0.2 * nm) # assumes that laser power is quickly attenuatted with R
print(fov)
print(f'power density for laser focused in FOV  = {power_density(laser,fov)/(W/cm2):5.1f} W/cm2')
I = photon_density(laser, fov, 1.0*second)
print(f'photon density for UV laser FOV (per pulse of 1s)  = {I/(1/cm2):5.1e} ph per cm2')
flm = fluorescence_per_molecule(m, I, 1*second)
print('Fluorescence per molecule and pulse 1 second = {0:5.1e}'.format(flm))

n_photons_reaching_CCD = flm * mc.transmission()
print('nof photons reaching CCD = {0:5.1e}'.format(n_photons_reaching_CCD))




        FOV; x = 8.0e-04 mm; y = 8.0e-04 mm; z = 2.0e-07 mm; area = 6.4e-07 mm2 volume = 1.3e-13 mm3
        
power density for laser focused in FOV  = 312500.0 W/cm2
photon density for UV laser FOV (per pulse of 1s)  = 4.2e+23 ph per cm2
Fluorescence per molecule and pulse 1 second = 3.0e+07
nof photons reaching CCD = 1.5e+06


In [94]:
lamda_e = 425 * nm 
n_photoelectrons_CCD       = n_photons_reaching_CCD * ccd.efficiency(lamda_e)
n_photoelectrons_pixel_CCD = photoelectrons_per_pixel(n_photoelectrons_CCD, ccd)

print('nof photons reaching CCD = {0:5.1e}'.format(n_photons_reaching_CCD))
print('nof photoelectrons CCD          = {0:5.1e}'.format(n_photoelectrons_CCD))
print('nof photoelectrons in FOV (10 x 10 pixels)          = {0:5.1e}'.format(n_photoelectrons_CCD/100))
print('nof photoelectrons per pixel    = {0:5.1e}'.format(n_photoelectrons_pixel_CCD))
print('S/N per pixel                   = {0:5.1e}'.format(np.sqrt(n_photoelectrons_pixel_CCD)))

nof photons reaching CCD = 1.5e+06
nof photoelectrons CCD          = 1.0e+06
nof photoelectrons in FOV (10 x 10 pixels)          = 1.0e+04
nof photoelectrons per pixel    = 4.0e+00
S/N per pixel                   = 2.0e+00


## Dry fluorescence experiments

- After completion of the wet experiments one could perform several dry experiments:

1. Observe SMFI in unchelated monolayer.
2. Observe SMFI in Barium-spiked (saturated) monolayer. 
3. Observe SMFI in monolayer + Ali drop.

- An important point to decide is whether (1) and (2) can be performed "in air" or need to be performed in vacuum. This depends of the effect on the monolayer of atmosphera. We assume that (3) will be performed in vacuum. 

## Differences between PVA substrates (wet) and monolayer (dry)

- The thickness of a monolayer is very thin. There are no "background molecules", like in the wet case. Therefore, TIRF is not needed. This is afortunate, on the other hand, since TIRF is not possible (no oil) in vacuum. One consequence is that the NA of the microscope will be smaller than one (assume 0.7)
- We need to decide whether to decouple the laser movement and focusing from the objective movement and focusing. Perhpas the simple thing is to couple both and iluminate from behind the sample.
- To perform the experiments we need a transparent substrate, either quartz or ITO. Perhaps quartz is best to start (we don't need an electric field yet), but ITO should also be investigated.
- The photon absorption cross section should be of the same order than Fluo-3

## SMFI in dry


In [12]:
dl = diffraction_limit(laser, mc) 
print(f'the difraction limit in nm is {dl/nm:5.1f}')

NameError: name 'laser' is not defined

#### With an amplification of 100 one could imagine a single molecule in one pixel. The difraction limit will be 10 x 10 mu after amplification, each pixel is 16 mu x 16 mu. Thus one consider a FOV of 160 x 160 nm, or in round number 200 nm. The numbers below show that we can do better than in the PRL, assuming that the cross sections and the quantum yield of the molecule are similar. 

In [8]:
fov = FoV(x = 0.2 * micron, y = 0.2 * micron, z = 1 * nm) # assumes that laser power is quickly attenuatted with R
print(fov)
print(f'power density for laser focused in FOV  = {power_density(luv,fov)/(W/cm2):5.1f} W/cm2')
I = photon_density(luv, fov, 0.5*second)
print(f'photon density for UV laser FOV (per pulse of 500 ms)  = {I/(1/cm2):5.1e} ph per cm2')
flm = fluorescence_per_molecule(m, I, 0.5*second)
print('Fluorescence per molecule and pulse (500 ms) second = {0:5.1e}'.format(flm))



        FOV; x = 2.0e-04 mm; y = 2.0e-04 mm; z = 1.0e-06 mm; area = 4.0e-08 mm2 volume = 4.0e-14 mm3
        


NameError: name 'luv' is not defined

In [ ]:
n_photons_reaching_CCD = flm * mc.transmission()
print('nof photoelectrons reaching CCD = {0:5.1e}'.format(n_photons_reaching_CCD))

n_photoelectrons_CCD       = n_photons_reaching_CCD * ccd.efficiency(laser.lamda)
n_photoelectrons_pixel_CCD = photoelectrons_per_pixel(n_photoelectrons_CCD, ccd)

print('nof photoelectrons CCD          = {0:5.1e}'.format(n_photoelectrons_CCD))
print('nof photoelectrons per pixel    = {0:5.1e}'.format(n_photoelectrons_pixel_CCD))
print('S/N per pixel                   = {0:5.1e}'.format(np.sqrt(n_photoelectrons_pixel_CCD)))

# TOPATU setup + Fuchsin 

## Diode laser
Beam size = ø 3.5 mm

In [25]:
weakLaser = Laser(lamda=532 * nm, power = 4.5 * milliwatt)
weakLaser


        Laser:
        wavelength                =5.3e+02 nm
        photon energy             =2.3e+00 eV
        power                     =4.5e+00 mW
        energy per second         =4.5e+00 mJ
        photons per second        =1.2e+16 ph/second
        

## Microscope
Dichroic cuts at 567 nm, with transmission efficiency of ~95%

https://www.thorlabs.com/newgrouppage9.cfm?objectgroup_id=3313&pn=DMLP567R

No filter, so eff_filter = 1

OB has NA = 0.3, M = 30x


In [15]:
mmc = Microscope(name='TOPATU', numerical_aperture=0.3, magnification=30, eff_filter=1, eff_dichroic=0.95)
print(mmc)


        name                 = TOPATU
        NA                   = 3.0e-01
        M                    = 3.0e+01
        eff dichroic         = 9.50e-01
        eff filter           = 1.00e+00
        eff PMT              = 3.00e-01
        Optical transmission = 2.23e-02
        Filter  transmission = 9.50e-01
        Total transmission   = 6.05e-03
        


### Using a better OB

NA = 0.85

M = 60x

In [43]:
mOB = Microscope(name='TOPATU_goodOB', numerical_aperture=0.85, magnification=60, eff_filter=1, eff_dichroic=0.95)
print(mOB)


        name                 = TOPATU_goodOB
        NA                   = 8.5e-01
        M                    = 6.0e+01
        eff dichroic         = 9.50e-01
        eff filter           = 1.00e+00
        eff PMT              = 3.00e-01
        Optical transmission = 1.70e-01
        Filter  transmission = 9.50e-01
        Total transmission   = 4.60e-02
        


In [13]:
dl = diffraction_limit(weakLaser, mmc) 
print(f'the difraction limit in nm is {dl/nm:5.1f}')

the difraction limit in nm is 1081.7


In [21]:
fov = FoV(x = 100 * micron, y = 100 * micron, z = 1 * nm) # assumes that laser power is quickly attenuatted with R
print(fov)
print(f'power density for laser focused in FOV  = {power_density(weakLaser,fov)/(W/cm2):5.1f} W/cm2')
I = photon_density(weakLaser, fov)
print(f'photon density for UV laser FOV (per pulse of 1 s)  = {I/(1/cm2):5.1e} ph per cm2')



        FOV; x = 1.0e-01 mm; y = 1.0e-01 mm; z = 1.0e-06 mm; area = 1.0e-02 mm2 volume = 1.0e-08 mm3
        
power density for laser focused in FOV  =  45.0 W/cm2
photon density for UV laser FOV (per pulse of 1 s)  = 1.2e+11 ph per cm2


## Calibration fluorophore: New Fuchsin
https://en.wikipedia.org/wiki/New_fuchsine

Not to be confused with *basic* fuchsin, however, could only find Q and $\sigma$ for basic fuchsin, assume they're similar

Single absorption cross section [https://core.ac.uk/download/pdf/11534085.pdf]

$\sigma^{(L)} = (4 \pm 2) \times 10^{-17}$

QY at C = 1e-5 M [https://link.springer.com/content/pdf/10.1007/s10895-014-1369-0.pdf] (figure 6. x axis scale is a typo)

Q = 0.4 

In [27]:
fuch = Molecule(name='Fuchsin', sigma=4E-17 * cm2, Q = 0.4)  
print(fuch)
flm = fluorescence_per_molecule(fuch, I)
print('Fluorescence per molecule = {0:5.1e}'.format(flm))


        Molecule name =Fuchsin; cross section = 4.0e-17 cm2; Q =   0.4
        
Fluorescence per molecule = 1.9e-06


## Detection
ORCA Flash V3.0: [https://www.hamamatsu.com/resources/pdf/sys/SCAS0134E_C13440-20CU_tec.pdf]

In [38]:
orca = CCD(name = 'C13440-20CU', n_pixels = (2048, 2048), 
           size_pixels=(6.5*micron, 6.5*micron),  effective_area=(13.312* mm, 13.312*mm),
          linear_full_well=(3e4, 150), pixel_clock_rate=(85 * MHZ, 85*MHZ),
          dark_current=0.06, readout_noise=1.0, readout_speed=100) 

In [41]:
n_photons_reaching_CCD = flm * mmc.transmission()
print('nof photoelectrons reaching CCD = {0:5.1e}'.format(n_photons_reaching_CCD))

n_photoelectrons_CCD       = n_photons_reaching_CCD * orca.efficiency(weakLaser.lamda)
n_photoelectrons_pixel_CCD = photoelectrons_per_pixel(n_photoelectrons_CCD, orca)

print('nof photoelectrons CCD          = {0:5.1e}'.format(n_photoelectrons_CCD))
print('nof photoelectrons per pixel    = {0:5.1e}'.format(n_photoelectrons_pixel_CCD))
print('S/N per pixel                   = {0:5.1e}'.format(np.sqrt(n_photoelectrons_pixel_CCD)))

nof photoelectrons reaching CCD = 1.2e-08
nof photoelectrons CCD          = 1.1e-08
nof photoelectrons per pixel    = 2.7e-15
S/N per pixel                   = 5.2e-08
